## Vacant Block Percent Land

In [ ]:
## load data
vacant_block = gpd.read_file('data/Vacant_Block_Percent_Land.geojson')

In [ ]:
sns.kdeplot(vacant_block['PARCELCOUNT'])

In [ ]:
vacant_block.plot()

In [ ]:
## aggregate by block group

# align the crs
vacant_block = vacant_block.to_crs(philly_2023.crs)

# check if the vacant block is fully contained in the philly 2023
vacant_block['fully_contained'] = vacant_block.geometry.apply(
    lambda geom: philly_2023.contains(geom).any()
)


In [ ]:
# Initialize a column to store the ID of the philly_2023 polygon assigned to each source polygon
vacant_block['assigned_philly_id'] = np.nan

# Iterate over source polygons
for idx, row in vacant_block.iterrows():
    geom = row.geometry
    
    # Check if fully contained
    containing = philly_2023[philly_2023.contains(geom)]
    if not containing.empty:
        vacant_block.at[idx, 'assigned_philly_id'] = containing.iloc[0].name
    else:
        # Calculate overlaps
        overlaps = philly_2023[philly_2023.intersects(geom)].copy()
        if not overlaps.empty:
            overlaps['intersection_area'] = overlaps.geometry.intersection(geom).area
            overlap_frac = overlaps['intersection_area'] / geom.area
            max_overlap = overlap_frac.max()
            if max_overlap > 0.5:
                assigned_id = overlaps.loc[overlap_frac.idxmax()].name
                vacant_block.at[idx, 'assigned_philly_id'] = assigned_id

In [ ]:
vacant_block = vacant_block.dropna(subset=['assigned_philly_id'])
vacant_block['assigned_philly_id'] = vacant_block['assigned_philly_id'].astype(int)

# Sum LANDVACCOUNT by assigned philly_2023 polygon
aggregated = vacant_block.groupby('assigned_philly_id')['LANDVACCOUNT'].sum()

# copy philly_2023
philly_2023_copy = philly_2023.copy()

# Merge back into philly_2023_gdf
philly_2023_copy['LANDVACCOUNT_sum'] = philly_2023_copy.index.map(aggregated).fillna(0)

In [ ]:
philly_2023_copy.head()

In [ ]:
sns.kdeplot(philly_2023_copy['LANDVACCOUNT_sum'])

In [ ]:
# load colormap
cmap = sns.cubehelix_palette(as_cmap=True)
#cmap = plt.cm.BuGn

norm = colors.Normalize(vmin=0, vmax=philly_2023_copy['LANDVACCOUNT_sum'].max())


# initialize the figure
fig, ax = plt.subplots(figsize=(7, 7))

# create the plot
philly_2023_copy.plot(ax=ax, column='LANDVACCOUNT_sum', cmap=cmap, edgecolor='gray', linewidth=0.5, alpha = 0.8,  legend=True)

